# Практическое задание к уроку 2


"Бейзлайны и детерминированные алгоритмы item-item"

In [1]:
# !pip install implicit

     |████████████████████████████████| 1.1 MB 1.5 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp39-cp39-macosx_10_9_x86_64.whl size=744366 sha256=41b61f2e12a51674a08b4e1e7644904fa5f0c5a15937e8304cc71fda1368dae7
  Stored in directory: /Users/tombelov/Library/Caches/pip/wheels/2d/0a/a4/6aea635157daf166f50276da2f2ceb04d17f5e37cb3f171ac7
Successfully built implicit


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender
from scipy.sparse import csr_matrix, coo_matrix

In [4]:
DATA_PATH = './retail_train.csv'

In [5]:
try:
    from collections import OrderedDict
except ImportError:
    from ordereddict import OrderedDict

### Загружаем данные

In [6]:
data = pd.read_csv(DATA_PATH)
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


### Исследуем исходные данные

In [7]:
data['week_no'].nunique()

95

In [8]:
users = data.user_id.nunique() 
items = data.item_id.nunique()
interactions = data.shape[0]

users, items, interactions

(2499, 89051, 2396804)

In [9]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [ ]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.describe()

,item_id,user_id
count,8.905100e+04,89051.000000
mean,5.115772e+06,14.759767
std,5.178973e+06,45.904111
min,2.567100e+04,1.000000
25%,9.665830e+05,1.000000
50%,1.448516e+06,2.000000
75%,9.553042e+06,10.000000
max,1.802456e+07,2039.000000


### Разбиваем массив на train и test выборки по времени (последний месяц)

In [10]:
test_size_weeks = 4

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [11]:
data_train.shape[0], data_test.shape[0]

(2251517, 145287)

# task2 Можно ли улучшить бейзлайны, если считать их на топ-5000 товарах?


data.item_id.nunique()

popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

len(top_5000)

data_train.head()

# Скопируем data_train, чтобы иметь возможность тестировать оба датасета.

data_train_5000 = data_train.copy(deep = True)

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train_5000.loc[~data_train_5000['item_id'].isin(top_5000), 'item_id'] = 999999

data_train.item_id.nunique()

data_train_5000.item_id.nunique()

## Создаем бейзлайны

In [12]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[829563, 830156, 832990, 840361, 856942, 87157..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [13]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2101 юзеров
В тестовом дата сете 0 новых юзеров


### Случайные рекомендации

In [15]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n+1, replace=False)

    if 999999 in recs:
      recs = np.setdiff1d(recs, [999999], assume_unique=True)

    
    return recs[:n].tolist()

In [16]:
result.head(2)

,user_id,actual
0,1,"[829563, 830156, 832990, 840361, 856942, 87157..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [17]:
items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

In [18]:
items

array([ 1004906,  1033142,  1036325, ..., 13850379, 15635984, 15982111])

### Рекомендации по популярности

In [19]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n+1).item_id

    if 999999 in recs:
      recs = np.setdiff1d(recs, [999999], assume_unique=True)
    #print(recs)
    return recs[:n].tolist()

In [20]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 91.4 ms, sys: 13.7 ms, total: 105 ms
Wall time: 106 ms


### Взвешенные случайные рекомендации (Weighted random recommendation)

In [21]:
def weighted_random_recommendation(items, n=5):
    items = np.array(items)
    recs = np.random.choice(items, size=n*2, replace=False)
    recs = np.unique(recs)[:n+1]

    if 999999 in recs:
      recs = np.setdiff1d(recs, [999999], assume_unique=True)
    
    return recs[:n].tolist()

In [24]:
%%time

items = data_train.item_id

result['weighted_random_recommendation'] = result['user_id'].apply(
    lambda x: weighted_random_recommendation(items, n=5)
)

CPU times: user 2min 29s, sys: 5.94 s, total: 2min 35s
Wall time: 2min 36s


In [26]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[829563, 830156, 832990, 840361, 856942, 87157...","[826860, 9420257, 1780146, 109081, 82699]","[6534178, 6533889, 1029743, 6534166, 1082185]","[854852, 867547, 879528, 883218, 933835]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1370778, 1000050, 1277455, 852112, 829303]","[6534178, 6533889, 1029743, 6534166, 1082185]","[847573, 862139, 985699, 999971, 1030182]"


### Рассчитаем метрику precision@5 и сравним модели

In [27]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[829563, 830156, 832990, 840361, 856942, 87157...","[826860, 9420257, 1780146, 109081, 82699]","[6534178, 6533889, 1029743, 6534166, 1082185]","[854852, 867547, 879528, 883218, 933835]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1370778, 1000050, 1277455, 852112, 829303]","[6534178, 6533889, 1029743, 6534166, 1082185]","[847573, 862139, 985699, 999971, 1030182]"


In [28]:
result.to_csv('predictions_basic.csv', index=False)

In [ ]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list) 
    
    return precision

In [30]:
result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean()

0.0008567348881485009

In [32]:
result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean()

0.16335078534031414

In [33]:
result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual']), axis=1).mean()

0.01980009519276535

### Популярные товары работают лучше )